In [1]:
!pip install jax

In [2]:
import jax.numpy as np
from jax import grad,jit,vmap
from jax import random
import matplotlib.pyplot as py
import numpy as onp
from jax import device_put
import random

In [3]:
!git clone https://github.com/pluu2/JaxNetwork

Cloning into 'JaxNetwork'...
remote: Enumerating objects: 46, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 46 (delta 9), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (46/46), done.


In [4]:
from JaxNetwork.model.jax_base_model import jax_base_model
from JaxNetwork.model.network_dense import network_dense


###Model 

  

In [5]:

class model(jax_base_model): 

  def __init__(self): 
    jax_base_model.__init__(self)
    self.layer.append(network_dense(2,10))
    self.layer.append(network_dense(10,4))
    self.layer.append(network_dense(4,1))


    self.num_layers=len(self.layer)
    self.input=[]
    self.target=[]

  def forward(self,x): 
    l1=self.layer[0].forward(x) 
    l1=self.tanh_act(l1) 
    l2=self.layer[1].forward(l1)
    l2=self.tanh_act(l2)
    l3=self.layer[2].forward(l2)
    l3=self.sigmoid_act(l3) 
    return l3

  
  def backwards (self,parameters): 
    l1=self.layer[0].forward(self.input,parameters[str(0) + ' weights'], parameters[str(0) + ' bias']) 
    l1=self.tanh_act(l1) 
    l2=self.layer[1].forward(l1,parameters[str(1) + ' weights'], parameters[str(1) + ' bias'] )
    l2=self.tanh_act(l2)
    l3=self.layer[2].forward(l2,parameters[str(2) + ' weights'], parameters[str(2) + ' bias'])
    l3=self.sigmoid_act(l3) 
    return self.binary_crossentropy(l3[0],self.target)

  

  

#Training

In [6]:
onp.random.seed(1000)
model1=model()
input_=[[1,0],
        [0,1],
        [1,1],
        [0,0]]
targets=[1.0,
         1.0,
         0.0,
         0.0]
input_=np.array(input_)

ctr=0
loss=0
for epochs in range (200):
  trainable_variables=model1.trainable_variables()
  model1.set_input(input_[ctr])
  model1.set_targets(targets[ctr])
  model_gradient=model1.gradient(trainable_variables)
  model1.update_weights(model_gradient,lr=0.1)
  loss+=model1.backwards(trainable_variables)
  ctr+=1
  if ctr>3:
    ctr=0
  if not epochs%100:
    print (round(model1.forward(input_)))

/usr/local/lib/python3.6/dist-packages/jax/lib/xla_bridge.py:127: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')
/usr/local/lib/python3.6/dist-packages/jax/lax/lax.py:5487: UserWarning: Explicitly requested dtype <class 'jax.numpy.lax_numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))


[[1]
 [1]
 [1]
 [0]]
[[1]
 [1]
 [0]
 [0]]


In [7]:
round(model1.forward([[1,0],[0,1],[1,1],[0,0]]))

/usr/local/lib/python3.6/dist-packages/jax/lax/lax.py:5487: UserWarning: Explicitly requested dtype <class 'jax.numpy.lax_numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  warnings.warn(msg.format(dtype, fun_name , truncated_dtype))


DeviceArray([[1],
             [1],
             [0],
             [0]], dtype=int32)